In [1]:
import spotipy
import spotipy.util as util
import os
from json import load

# make the notebook automatically reload external python modules
%load_ext autoreload
%autoreload 2

In [2]:
with open('credentials.txt') as fd:
    creds = load(fd)

In [3]:
# For the purposes of this notebook we only want to read from a users library
scope = 'user-library-read'

In [4]:
def get_token(scope, **kwargs):
    try:
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    except:
        os.remove(f".cache-{kwargs['username']}")
        token = util.prompt_for_user_token(**kwargs, scope=scope)
    return token

In [5]:
token = get_token(scope, **creds)

In [6]:
sp = spotipy.Spotify(auth=token)

In [8]:
# some of the methods require your user id so its a good idea to save it now 
user = sp.current_user()
user_id = user['id']

In [10]:
def get_playlist_track_names_ids(playlist_id):
    """
    This function returns two lists where the first is the names
    of every track and the second are the associated track ids. 
    """
    global user_id
    playlist_info = sp.user_playlist(user_id, playlist_id=playlist_id)
    tracks = playlist_info['tracks']['items']
    names, ids = [], []
    
    for track in tracks:
        names.append(track['track']['name'])
        ids.append(track['track']['id'])
    
    return names, ids

In [11]:
def get_album_track_names_ids(album_id):
    """
    This function returns two lists where the first has the names 
    of every track and the second are the associated track ids. 
    """
    album = sp.album_tracks(album_id)
    names, ids = [], []
    
    for track in album['items']:
        names.append(track['name'])
        ids.append(track['id'])
    
    return names, ids

In [22]:
# Get all the track names and ids from the Chill Hits playlist
names, ids = get_playlist_track_names_ids('spotify:playlist:37i9dQZF1DX4WYpdgoIcn6')

## Reading all of a users playlists

## Creating a Playlist